In [1]:
!pip install python-telegram-bot nest_asyncio yfinance --quiet

## Подключение логгирования

In [3]:
import logging
import sys

def setup_logger(logfile='bot.log'):
    # Создаём логгер
    logger = logging.getLogger('stockbot')
    logger.setLevel(logging.DEBUG)  # Можно INFO, WARNING, ERROR

    # Формат логов
    formatter = logging.Formatter('%(asctime)s [%(levelname)s] %(message)s')

    # Хэндлер для файла
    fh = logging.FileHandler(logfile, encoding='utf-8')
    fh.setLevel(logging.DEBUG)
    fh.setFormatter(formatter)
    logger.addHandler(fh)

    # Хэндлер для консоли (stdout, чтобы видеть в ноутбуке)
    ch = logging.StreamHandler(sys.stdout)
    ch.setLevel(logging.INFO)
    ch.setFormatter(formatter)
    logger.addHandler(ch)

    return logger

logger = setup_logger()

## Работа с API Yahoo Finance

In [5]:
import os

tickets_path = os.path.join('./', 'yahoo-cache')

def get_ticket_filepath(ticker):
    return os.path.join(tickets_path, f"{ticker}.csv")

In [35]:
import yfinance as yf

def is_valid_ticker(ticker):
    try:
        hist = yf.download(ticker, period="1mo")
        logger.info(f"Проверка тикера {ticker}: найдено строк {len(hist)}")
        return not hist.empty
    except Exception as e:
        return False

In [7]:
import pandas as pd
from datetime import datetime, timedelta

def clean_ticker_csv(filename):
    df = pd.read_csv(filename, skiprows=2)
    df.columns = ['Date', 'close', 'high', 'low', 'open', 'volume']
    df.to_csv(filename, index=False)
    return df

def save_ticker_history(ticker):
    end_date = datetime.today()
     # 2 года
    start_date = end_date - timedelta(days=730)

    ticker_obj = yf.Ticker(ticker)
    hist = yf.download(ticker, start=start_date.strftime('%Y-%m-%d'), end=end_date.strftime('%Y-%m-%d'))

    hist.to_csv(get_ticket_filepath(ticker))
    clean_ticker_csv(get_ticket_filepath(ticker))
    return filename

In [8]:
def file_exists(ticker):
    return os.path.isfile(get_ticket_filepath(ticker))

## Анализ файла с тикером

In [33]:
print(is_valid_ticker('AAPL'))  # True
print(is_valid_ticker('NVDA'))  # True
print(is_valid_ticker('FAKETICKER'))  # False

False
False
False


## Telegram bot

In [11]:
from telegram import Update
from telegram.ext import Application, CommandHandler, MessageHandler, ContextTypes, filters

In [12]:
TELEGRAM_TOKEN = '8482085191:AAH5khDSXWXwDB02Gp5157bSennnXkT3Vog'

In [13]:
async def start(update: Update, context: ContextTypes.DEFAULT_TYPE):
    await update.message.reply_text(
        "Привет! Я бот для анализа и прогнозирования американских акций.\n"
        "Введите тикер компании (например, AAPL) и сумму для инвестиций в долларах через пробел.\n"
        "Пример 1: AAPL 1000\n"
        "Пример 2: NVDA 325"
    )

async def handle_message(update: Update, context: ContextTypes.DEFAULT_TYPE):
    try:
        text = update.message.text.strip()
        ticker, amount = text.split(' ')
        ticker = ticker.upper()
        # семантическая валидация ввода    
        try:
            amount = int(amount)
        except ValueError as e:
            await update.message.reply_text(f"Проверьте формат: не получилось перевести в число '{amount}'")
            return 
        # логическая валидация ввода + сохранение данных
        if is_valid_ticker(ticker):
            if not file_exists(ticker):
                save_ticker_history(ticker)
                print(f"Данные для {ticker} сохранены.")
            else:
                print(f"Файл для {ticker} уже существует.")
        else:
            await update.message.reply_text(f"Тикет '{ticker}' не найден")
            return 
        
        await update.message.reply_text(f"Тикет: {ticker}\nСумма денег: {amount}$")

    except Exception as e:
        logger.exception(f"Непредвиденная ошибка в handle_message: {e}")
        await update.message.reply_text("Произошла ошибка")
        
   
async def unknown(update: Update, context: ContextTypes.DEFAULT_TYPE):
    await update.message.reply_text("Извините, я не знаю такой команды. Введите тикер и сумму.")

In [39]:
# для предотвращения ошибки
# "This event loop is already running"
import nest_asyncio
nest_asyncio.apply()
import asyncio

from telegram.error import TelegramError

async def error_handler(update, context):
    logger.error(msg="Exception while handling an update:", exc_info=context.error)

async def on_shutdown(application):
    logger.info("Бот остановлен (выключен)")

async def main_async():
    try:
        logger.info("Бот запускается...")
        app = Application.builder().token(TELEGRAM_TOKEN).build()
    
        app.add_handler(CommandHandler("start", start))
        app.add_handler(MessageHandler(filters.TEXT & (~filters.COMMAND), handle_message))
        app.add_handler(MessageHandler(filters.COMMAND, unknown))
    
        app.add_error_handler(error_handler)
    
        logger.info("Бот запущен!")
        await app.run_polling()
    except Exception as e:
        logger.exception(f"Ошибка при запуске main_async: {e}")
    finally:
        logger.info("Бот остановлен (выключен)")     

task = asyncio.create_task(main_async())

2026-01-09 15:35:12,804 [INFO] Бот запускается...
2026-01-09 15:35:12,853 [INFO] Бот запущен!


In [37]:
task.cancel()

True